In [1]:
import numpy as np
import pandas as pd
from sympy import diff, lambdify, symbols, log, sqrt, exp

writer = pd.ExcelWriter('./Table 5 (0.5 year).xlsx')

beta = 0.11
alpha = 0.07
kappa = 0.38
mu = 0.07
omega = 0.81
rho = 0.10
T = 1/2
r0 = 0.06
V0 = 0.005
# symbolic variables
r, v, t, v0 = symbols('r v t v0')
A = (1 - exp(-beta * t)) / beta
D = (alpha - v / (2 * beta**2)) * (A - t) - v * A**2 / (4 * beta)
price_vasicek = exp(-A * r + D)
delta0 = (v * r - v0) / 2 * A**2 * price_vasicek

def price(R, V, T):
  A = (1 - np.exp(-beta * T)) / beta
  D = (alpha - V / (2 * beta**2)) * (A - T) - V * A**2 / (4 * beta)
  return np.exp(-A * R + D)

def L(f): # infinitesimal generator
  ft = diff(f, t)
  fr = diff(f, r)
  fr2 = diff(fr, r)
  fv = diff(f, v)
  fv2 = diff(fv, v)
  fvr = diff(fv, r)
  tmp1 = -ft + beta * (alpha - r) * fr
  tmp2 = v**2 * (r * fr2 + omega**2 * fv2) / 2
  tmp3 = rho * v**2 * sqrt(r) * omega * fvr
  return tmp1 + tmp2 + tmp3 - r * f

In [2]:
delta = [delta0] # symbolic equations of delta terms
delta_func = []
for _ in range(1, 5):
  delta.append(L(delta[-1]))
for i in range(0, 5):
  delta_func.append(lambdify([r, v, t, v0], delta[i]))

In [3]:
%%time
R = np.arange(0.04, 0.09, 0.01)
Approx_4_r = []
for r in R:
  p0_approx = price(r, V0, T) + T * delta_func[0](r, V0, T, V0*r)
  p1_approx = p0_approx + T**2 / 2 * delta_func[1](r, V0, T, V0*r)
  p2_approx = p1_approx + T**3 / 6 * delta_func[2](r, V0, T, V0*r)
  p3_approx = p2_approx + T**4 / 24 * delta_func[3](r, V0, T, V0*r)
  p4_approx = p3_approx + T**5 / 120 * delta_func[4](r, V0, T, V0*r)
  Approx_4_r.append(p4_approx)

Wall time: 3.41 s


In [4]:
Price_MC_r = np.array([0.9798, 0.9750, 0.9703, 0.9656, 0.9609])
Price_JFE_r = np.array([0.9797, 0.9750, 0.9703, 0.9656, 0.9609])
Diff_JFE_r = np.array([-0.0010, -0.0011, -0.0012, -0.0013, -0.0014])
Diff_replicate_r = (Approx_4_r - Price_MC_r) / Price_MC_r * 100

df1 = pd.DataFrame({
  "R": R,
  "MC price": Price_MC_r,
  "Replicate price": np.round(Approx_4_r, 4),
  "Replicate %diff": np.round(Diff_replicate_r, 4),
  "JFE price": Price_JFE_r,
  "JFE %diff": Diff_JFE_r
  })
print(df1)

      R  MC price  Replicate price  Replicate %diff  JFE price  JFE %diff
0  0.04    0.9798           0.9799           0.0102     0.9797    -0.0010
1  0.05    0.9750           0.9751           0.0148     0.9750    -0.0011
2  0.06    0.9703           0.9704           0.0115     0.9703    -0.0012
3  0.07    0.9656           0.9657           0.0106     0.9656    -0.0013
4  0.08    0.9609           0.9610           0.0120     0.9609    -0.0014


In [5]:
%%time
V = np.arange(0.004, 0.0065, 0.0005)
Approx_4_v = []
for v in V:
  p0_approx = price(r0, v, T) + T * delta_func[0](r0, v, T, v*r0)
  p1_approx = p0_approx + T**2 / 2 * delta_func[1](r0, v, T, v*r0)
  p2_approx = p1_approx + T**3 / 6 * delta_func[2](r0, v, T, v*r0)
  p3_approx = p2_approx + T**4 / 24 * delta_func[3](r0, v, T, v*r0)
  p4_approx = p3_approx + T**5 / 120 * delta_func[4](r0, v, T, v*r0)
  Approx_4_v.append(p4_approx)

Wall time: 3.44 s


In [6]:
Price_MC_v = np.array([0.9702, 0.9703, 0.9703, 0.9703, 0.9703])
Price_JFE_v = np.array([0.9702, 0.9703, 0.9703, 0.9703, 0.9703])
Diff_JFE_v = np.array([-0.0008, -0.0009, -0.0009, -0.0010, -0.0010])
Diff_replicate_v = (Approx_4_v - Price_MC_v) / Price_MC_v * 100

df2 = pd.DataFrame({
  "sigma**2": V,
  "MC price": Price_MC_v,
  "Replicate price": np.round(Approx_4_v, 4),
  "Replicate %diff": np.round(Diff_replicate_v, 4),
  "JFE price": Price_JFE_v,
  "JFE %diff": Diff_JFE_v
})
print(df2)

   sigma**2  MC price  Replicate price  Replicate %diff  JFE price  JFE %diff
0    0.0040    0.9702           0.9704           0.0198     0.9702    -0.0008
1    0.0045    0.9703           0.9704           0.0105     0.9703    -0.0009
2    0.0050    0.9703           0.9704           0.0115     0.9703    -0.0009
3    0.0055    0.9703           0.9704           0.0125     0.9703    -0.0010
4    0.0060    0.9703           0.9704           0.0135     0.9703    -0.0010


In [7]:
df1.to_excel(writer, sheet_name="sigma_2=0.50%")
df2.to_excel(writer, sheet_name="r=6%")
writer.save()
writer.close()